## Import Modules

# M2 Rigid Body Position
 
This Jupyter notebook performs the Rigid Body Position (RBP) repeatability test of the M2.
Each M2 RBP Degree of Freedom (DoF) is actuated individually with a sequence of 7 commands of different amplitude.
Each commanded position is hold for 37s and each sequence of commands for a specific DoF is repeated for 7 times. 

In [ ]:
import asyncio
import yaml
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK
from lsst.ts.ofc.utils import get_config_dir as get_config_dir_ofc

## Declaration of User-defined Functions

In [ ]:
async def RBP_linear_displacement(csc, axis, value, linear_motion_time, time_hold_position=37):
    """Apply the RBP linear displacement to X, Y, Z DoFs

    Parameters
    ----------
    csc : `lsst.ts.salobj.remote.Remote`
        Remote object of the M2 CSC.
    axis : `string`
        axis of RBP selected.
    value : `numpy.ndarray`
        RBP displacement to apply (micron).
    motion_time : `float`
        Time to reach position in second, to be fine tuned empirically
    time_hold_position : `float`, optional
        Time to apply the hold the position in second. (the default is 5.0)""" 
 

    
    print(f"Apply the RBP linear displacement: {displacement} micron.")
    if axis == "x":
        await csc.cmd_positionMirror.set_start(x=value)
    elif axis == "y":
        await csc.cmd_positionMirror.set_start(y=value)
    elif axis == "z":
        await csc.cmd_positionMirror.set_start(z=value)
    await asyncio.sleep(linear_motion_time)
    await asyncio.sleep(time_hold_position)
    

In [ ]:
async def RBP_angular_displacement(csc, axis, value, angular_motion_time, time_hold_position=37):
    """Apply the RBP angular displacement to RX, RY, RZ DoFs

    Parameters
    ----------
    csc : `lsst.ts.salobj.remote.Remote`
        Remote object of the M2 CSC.
    axis : `string`
        axis of RBP selected.
    value : `numpy.ndarray`
        RBP angular displacement to apply (arcsec).
    motion_time : `float`
        Time to reach position in second, to be fine tuned empirically
    time_hold_position : `float`, optional
        Time to apply the hold the position in second. (the default is 5.0)""" 
 

    
    print(f"Apply the RBP angular displacement: {angular_displacement} arcsec.")
    if axis == "xRot":
        await csc.cmd_positionMirror.set_start(xRot=value)
    elif axis == "yRot":
        await csc.cmd_positionMirror.set_start(yRot=value)
    elif axis == "zRot":
        await csc.cmd_positionMirror.set_start(zRot=value)
    await asyncio.sleep(angular_motion_time)
    await asyncio.sleep(time_hold_position)
    
    

## Prepare the M2 CSC and put to Enabled state

In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=60)

## RBP linear displacement

In [ ]:
# displacement times to reach linear RBP in sec

linear_motion_time = np.array((65, 35, 20, 35, 35, 20, 30, 65))
print(linear_motion_time)

In [ ]:
# RBP linear displacement in microns applied to X Y Z DoFs including homing position

displacement = np.array((-250, -150, -100, 0, 100, 150, 250, 0))
print(displacement)

In [ ]:
# displacement times to reach angular RBP in sec

angular_motion_time = np.array((20, 15, 10, 10, 10, 15, 20, 20))
print(angular_motion_time)

In [ ]:
# RBP angular displacement in acrsec applied to RX RY RZ DoFs including homing position

angular_displacement = np.array((-15, -10, -5, 0, 5, 10, 15, 0))
print(angular_displacement)

## Apply linear displacement sequence of RBP

In [ ]:
# Try first one move on one DoF

time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


positions_dof = 1
repetition = 1

#*********************#
#                     #
#        X DOF        #
#                     #
#*********************#

axis = "x"
# repeat the DoFs command sequence for 1 times
for jdx in range(repetition):
    print(f"Repetition sequence number: {jdx+1} -------------------------------------")

# first DoF move of RBP commands
    for idx in range(positions_dof):
        #print(f"DoF: {idx}.")
        print(f"Status of advancement: DoF {axis}, Sequence:{jdx+1}, Commanded position micron: {displacement[idx]}")
        await RBP_linear_displacement(csc, "x", displacement[idx], linear_motion_time[idx], time_hold_position=37)
     
    
positions_dof = 1

# repeat the DoFs command sequence for 1 time
for jdx in range(repetition):
    print(f"Repetition sequence number: {jdx+1} -------------------------------------")

# come back from first DoF move of RBP commands
    for idx in range(7,8,positions_dof):
        #print(f"DoF: {idx}.")
        print(f"Status of advancement: DoF {axis}, Sequence:{jdx+1}, Commanded position micron: {displacement[idx]}")
        await RBP_linear_displacement(csc, "x", displacement[idx], linear_motion_time[idx], time_hold_position=37)
      
    
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")    

In [ ]:
#apply linear displacement sequence to X-DOF

time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


positions_dof = 8
repetition = 5

#*********************#
#                     #
#        X DOF        #
#                     #
#*********************#

axis = "x"
# repeat the DoFs command sequence for 5 times
for jdx in range(repetition):
    print(f"Repetition sequence number: {jdx+1}, -------------------------------------")

# DoF positions of RBP commands
    for idx in range(positions_dof):
        #print(f"DoF: {idx}.")
        print(f"Status of advancement: motion time, {linear_motion_time[idx]}, DoF {axis}, Sequence:{jdx+1}, Commanded position micron: {displacement[idx]}")
        await RBP_linear_displacement(csc, "x", displacement[idx], linear_motion_time[idx], time_hold_position=37)
        
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")        

In [ ]:
#apply linear displacement sequence Y-DOF
     
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


positions_dof = 8
repetition = 5

#*********************#
#                     #
#        Y DOF        #
#                     #
#*********************#

axis = "y"
# repeat the DoFs command sequence for 5 times
for jdx in range(repetition):
    print(f"Repetition sequence number: {jdx+1} -------------------------------------")

# DoF positions of RBP commands
    for idx in range(positions_dof):
        print(f"Status of advancement: DoF {axis}, Sequence:{jdx+1}, Commanded position micron: {displacement[idx]}")
        await RBP_linear_displacement(csc, "y", displacement[idx], linear_motion_time[idx], time_hold_position=37)
        
        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")        

In [ ]:
#apply linear displacement sequence Z-DOF

time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

positions_dof = 8
repetition = 5

#*********************#
#                     #
#        Z DOF        #
#                     #
#*********************#

axis = "z"
# repeat the DoFs command sequence for 5 times
for jdx in range(repetition):
    print(f"Repetition sequence number: {jdx+1} -------------------------------------")

# DoF positions of RBP commands
    for idx in range(positions_dof):
        print(f"Status of advancement: DoF {axis}, Sequence:{jdx+1}, Commanded position micron: {displacement[idx]}")
        await RBP_linear_displacement(csc, "z", displacement[idx], linear_motion_time[idx], time_hold_position=37)

        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")        

## Apply angular displacement sequence of RBP

In [ ]:
#apply angular displacement sequence RX-DOF

time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


positions_dof = 8
repetition = 5


#*********************#
#                     #
#       RX DOF        #
#                     #
#*********************#

axis = "xRot"
# repeat the DoFs command sequence for 5 times
for jdx in range(repetition):
    print(f"Repetition sequence number: {jdx+1} -------------------------------------")

# DoF positions of RBP commands
    for idx in range(positions_dof):
        #print(f"DoF: {idx}.")
        print(f"Status of advancement: DoF {axis}, Sequence:{jdx+1}, Commanded position arcsec: {angular_displacement[idx]}")
        await RBP_angular_displacement(csc, "xRot", angular_displacement[idx], angular_motion_time[idx], time_hold_position=37)

time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

In [ ]:
#apply angular displacement sequence RY-DOF


time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


positions_dof = 8
repetition = 5

        
#*********************#
#                     #
#       RY DOF        #
#                     #
#*********************#

axis = "yRot"
# repeat the DoFs command sequence for 5 times
for jdx in range(repetition):
    print(f"Repetition sequence number: {jdx+1} -------------------------------------")

# DoF positions of RBP commands
    for idx in range(positions_dof):
        print(f"Status of advancement: DoF {axis}, Sequence:{jdx+1}, Commanded position arcsec: {angular_displacement[idx]}")
        await RBP_angular_displacement(csc, "yRot", angular_displacement[idx], angular_motion_time[idx], time_hold_position=37)
    
    
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")    

In [ ]:
#apply angular displacement sequence RZ-DOF


time_start = datetime.now()
print(f"UTC time to is {time_start} now.")


positions_dof = 8
repetition = 5


#*********************#
#                     #
#       RZ DOF        #
#                     #
#*********************#

axis = "zRot"
# repeat the DoFs command sequence for 5 times
for jdx in range(repetition):
    print(f"Repetition sequence number: {jdx+1} -------------------------------------")

# DoF positions of RBP commands
    for idx in range(positions_dof):
        print(f"Status of advancement: DoF {axis}, Sequence:{jdx+1}, Commanded position arcsec: {angular_displacement[idx]}")
        await RBP_angular_displacement(csc, "zRot", angular_displacement[idx], angular_motion_time[idx], time_hold_position=37)

        
time_end = datetime.now()
print(f"UTC time to is {time_end} now.")        